In [ ]:
##################################### KERNEL#1
import os
from glob import glob
import re
import ast
import numpy as np 
import pandas as pd
import cv2
import urllib
from tqdm import tqdm
from dask import bag, threaded
from dask.diagnostics import ProgressBar
import gc

import matplotlib
import matplotlib.pyplot as pltc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import os
from keras.callbacks import EarlyStopping, ModelCheckpoint

from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import random

print(os.listdir("../"))


In [ ]:
##################################### KERNEL#2
# Read the strokes .csv files.

traindir = "../input/train_simplified/"

train_dataset = {}
train_images = 0

k = 0
country_list = set()
#showcase = []
drawlist = []
for file in os.listdir(traindir):
    k += 1
    # Iterate through all categories.
    if not file.endswith(".csv"):
        continue
    
    #print("Current file is: " + file)
    filename = file.split('.')[0]
    draw_df = None
    draw_df = pd.read_csv(traindir + file, nrows=500)

    for i in range(0, len(draw_df)):
        country_list.add(draw_df.countrycode[0])
        if(draw_df.recognized[i] == True):
            #print(draw_df.recognized[i])
            drawlist.append(draw_df)

In [ ]:
##################################### KERNEL#3
# A JSON containing a Class and a Number of images belonging to the Class.

{
   "airplane":151623,
   "alarm clock":123399,
   "ambulance":148004,
   "angel":149736,
   "animal migration":137847,
   "ant":124612,
   "anvil":126231,
   "apple":144722,
   "arm":120951,
   "asparagus":168102,
   "axe":124122,
   "backpack":125801,
   "banana":307936,
   "bandage":147614,
   "barn":151139,
   "baseball bat":123809,
   "baseball":135375,
   "basket":118458,
   "basketball":133793,
   "bat":118114,
   "bathtub":174336,
   "beach":124938,
   "bear":134762,
   "beard":165202,
   "bed":113862,
   "bee":120890,
   "belt":191119,
   "bench":128695,
   "bicycle":126527,
   "binoculars":124190,
   "bird":133572,
   "birthday cake":144982,
   "blackberry":128153,
   "blueberry":127878,
   "book":119364,
   "boomerang":142682,
   "bottlecap":153207,
   "bowtie":130283,
   "bracelet":119416,
   "brain":143033,
   "bread":120570,
   "bridge":133010,
   "broccoli":132826,
   "broom":116927,
   "bucket":124064,
   "bulldozer":187409,
   "bus":166208,
   "bush":120520,
   "butterfly":117999,
   "cactus":131676,
   "cake":124905,
   "calculator":128375,
   "calendar":321981,
   "camel":121399,
   "camera":128772,
   "camouflage":172710,
   "campfire":133395,
   "candle":141545,
   "cannon":141394,
   "canoe":123767,
   "car":182764,
   "carrot":132459,
   "castle":122534,
   "cat":123202,
   "ceiling fan":115413,
   "cell phone":121130,
   "cello":149725,
   "chair":222706,
   "chandelier":167502,
   "church":164225,
   "circle":122876,
   "clarinet":126214,
   "clock":120536,
   "cloud":120265,
   "coffee cup":183432,
   "compass":127609,
   "computer":123885,
   "cookie":131353,
   "cooler":271444,
   "couch":119662,
   "cow":123083,
   "crab":126930,
   "crayon":129953,
   "crocodile":127932,
   "crown":134089,
   "cruise ship":123410,
   "cup":130721,
   "diamond":131587,
   "dishwasher":169759,
   "diving board":290239,
   "dog":152159,
   "dolphin":121613,
   "donut":140751,
   "door":120230,
   "dragon":124362,
   "dresser":123395,
   "drill":136786,
   "drums":137299,
   "duck":135480,
   "dumbbell":157975,
   "ear":122897,
   "elbow":126253,
   "elephant":126969,
   "envelope":134863,
   "eraser":118339,
   "eye":125888,
   "eyeglasses":225762,
   "face":161666,
   "fan":136158,
   "feather":119910,
   "fence":129426,
   "finger":167957,
   "fire hydrant":137242,
   "fireplace":155570,
   "firetruck":220695,
   "fish":134150,
   "flamingo":124569,
   "flashlight":239763,
   "flip flops":121518,
   "floor lamp":166355,
   "flower":144818,
   "flying saucer":151966,
   "foot":203086,
   "fork":126077,
   "frog":159047,
   "frying pan":123824,
   "garden hose":121843,
   "garden":158527,
   "giraffe":127182,
   "goatee":190002,
   "golf club":194843,
   "grapes":155305,
   "grass":123071,
   "guitar":120451,
   "hamburger":129672,
   "hammer":119012,
   "hand":291773,
   "harp":285403,
   "hat":222610,
   "headphones":118906,
   "hedgehog":120527,
   "helicopter":159938,
   "helmet":121899,
   "hexagon":142435,
   "hockey puck":203301,
   "hockey stick":130110,
   "horse":178286,
   "hospital":167448,
   "hot air balloon":126350,
   "hot dog":181999,
   "hot tub":120279,
   "hourglass":135957,
   "house plant":122996,
   "house":135420,
   "hurricane":136245,
   "ice cream":123133,
   "jacket":214124,
   "jail":120131,
   "kangaroo":174470,
   "key":160965,
   "keyboard":187766,
   "knee":267540,
   "ladder":125389,
   "lantern":149912,
   "laptop":261501,
   "leaf":125571,
   "leg":116804,
   "light bulb":120879,
   "lighthouse":160903,
   "lightning":151560,
   "line":143549,
   "lion":120949,
   "lipstick":127623,
   "lobster":140175,
   "lollipop":128849,
   "mailbox":130053,
   "map":120629,
   "marker":319136,
   "matches":143969,
   "megaphone":137334,
   "mermaid":180304,
   "microphone":120570,
   "microwave":130533,
   "moon":121661,
   "mosquito":123029,
   "motorbike":169931,
   "mountain":128540,
   "mouse":178826,
   "moustache":179924,
   "mouth":134135,
   "mug":152918,
   "mushroom":142167,
   "nail":158593,
   "necklace":120580,
   "nose":197573,
   "ocean":131493,
   "octagon":159474,
   "octopus":150152,
   "onion":132297,
   "oven":206910,
   "owl":169632,
   "paint can":123446,
   "paintbrush":187002,
   "palm tree":121959,
   "panda":113613,
   "pants":144264,
   "paper clip":127129,
   "parachute":127319,
   "parrot":185530,
   "passport":150265,
   "peanut":126626,
   "pear":116904,
   "peas":161656,
   "pencil":122001,
   "penguin":253791,
   "piano":116870,
   "pickup truck":130740,
   "picture frame":122371,
   "pig":186770,
   "pillow":118753,
   "pineapple":125071,
   "pizza":130371,
   "pliers":172549,
   "police car":130024,
   "pond":121620,
   "pool":133439,
   "popsicle":126707,
   "postcard":125706,
   "potato":329204,
   "power outlet":169462,
   "purse":123320,
   "rabbit":155288,
   "raccoon":119588,
   "radio":135728,
   "rain":134680,
   "rainbow":126845,
   "rake":154639,
   "remote control":119644,
   "rhinoceros":188484,
   "river":133271,
   "roller coaster":143570,
   "rollerskates":119772,
   "sailboat":136506,
   "sandwich":131732,
   "saw":121256,
   "saxophone":118107,
   "school bus":122041,
   "scissors":123390,
   "scorpion":165689,
   "screwdriver":116313,
   "sea turtle":119876,
   "see saw":131936,
   "shark":126050,
   "sheep":126121,
   "shoe":120231,
   "shorts":124970,
   "shovel":117194,
   "sink":208410,
   "skateboard":128733,
   "skull":126174,
   "skyscraper":183709,
   "sleeping bag":119691,
   "smiley face":124386,
   "snail":133757,
   "snake":122273,
   "snorkel":154533,
   "snowflake":116685,
   "snowman":340029,
   "soccer ball":125349,
   "sock":205715,
   "speedboat":188580,
   "spider":209447,
   "spoon":125028,
   "spreadsheet":170200,
   "square":125145,
   "squiggle":118441,
   "squirrel":156883,
   "stairs":128981,
   "star":137619,
   "steak":122042,
   "stereo":122444,
   "stethoscope":153794,
   "stitches":125192,
   "stop sign":119814,
   "stove":116535,
   "strawberry":122301,
   "streetlight":123280,
   "string bean":119083,
   "submarine":124362,
   "suitcase":126442,
   "sun":133781,
   "swan":152088,
   "sweater":120184,
   "swing set":119357,
   "sword":123802,
   "t-shirt":125233,
   "table":128021,
   "teapot":126804,
   "teddy-bear":179568,
   "telephone":127885,
   "television":123137,
   "tennis racquet":231151,
   "tent":131527,
   "The Eiffel Tower":134801,
   "The Great Wall of China":193015,
   "The Mona Lisa":121383,
   "tiger":121067,
   "toaster":122434,
   "toe":153652,
   "toilet":129888,
   "tooth":125064,
   "toothbrush":124828,
   "toothpaste":131037,
   "tornado":143271,
   "tractor":116677,
   "traffic light":125321,
   "train":127948,
   "tree":144721,
   "triangle":123170,
   "trombone":184759,
   "truck":131354,
   "trumpet":169547,
   "umbrella":124084,
   "underwear":124548,
   "van":165909,
   "vase":126475,
   "violin":217260,
   "washing machine":120851,
   "watermelon":132939,
   "waterslide":185364,
   "whale":116502,
   "wheel":136659,
   "windmill":120644,
   "wine bottle":126373,
   "wine glass":132302,
   "wristwatch":162645,
   "yoga":280442,
   "zebra":144608,
   "zigzag":120072
}

In [ ]:
##################################### KERNEL#4
# This kernel splits the data into train and val set.

def empty_folder(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)

    

class_files = os.listdir('../input/train_simplified/')
class_dict = {x[:-4].replace(" ", "_"):i for i, x in enumerate(class_files)}
reverse_dict = {v: k for k, v in class_dict.items()}

img_rows, img_cols = 28, 28
ims_per_class = 100

num_classes = len(class_files)

def get_ims(strokes): # Get image out of strokes.
    strokes = ast.literal_eval(strokes) 
    fig, ax = plt.subplots()
    for x,y in strokes:
        ax.plot(x, y, linewidth=12.)
    ax.axis('off')
    
    #### DATA Augumentation - Random erasing
    erasing_probability = 0
    rand_probability = random.uniform(0, 1)
    if rand_probability < erasing_probability:
        # Let's erase something
        random_width = random.randint(20,50)
        random_height = random.randint(20,50) 
        someX, someY = 100, 0.5
        randomX = random.randint(1,255)
        randomY = random.randint(1,60)
        currentAxis = plt.gca()
        currentAxis.add_patch(Rectangle((randomX, randomY), random_width, random_height, facecolor="white", alpha=1))
    
    fig.canvas.draw()
    X = np.array(fig.canvas.renderer._renderer)
    plt.close("all")
    plt.clf()
    X = (cv2.resize(X, (img_rows, img_cols)) / 255.)[::-1]
    X = X[:, :, 3]
    return X

# get train arrays
train_grandarray = np.empty((num_classes, ims_per_class, img_rows, img_cols))
labelarray = np.empty((num_classes, ims_per_class))

class_paths = glob('../input/train_simplified/*.csv')
lm = 0
for i,c in enumerate(tqdm(class_paths[0: num_classes])):
    train = pd.read_csv(c, usecols=['word', 'drawing', 'countrycode'], nrows=ims_per_class)
    train['word'] = train.word.str.replace(" ", "_")
    for m in range(0, len(train.drawing)):
        lm += 1
    imagebag = bag.from_sequence(train.drawing.values).map(get_ims)
    trainarray = np.array(imagebag.compute())
    train_grandarray[i] = trainarray
    label = train.word.map(class_dict)
    labelarray[i] = label

labelarray = np.expand_dims(np.concatenate(labelarray), 1)
train_grandarray = np.concatenate(train_grandarray)
train_grandarray = np.reshape(train_grandarray, (ims_per_class*num_classes, -1))
train_grandarray = np.concatenate((labelarray, train_grandarray), axis=1)
#labelarray

In [ ]:
##################################### KERNEL#5
# This kernel prepares the test dataset.

if not os.path.exists('../my/test'):
    #empty_folder('../my/test/')
    os.makedirs('../my/test')
    
if not os.path.exists('../my/checkpoint'):
    os.makedirs('../my/checkpoint')

class_paths = glob('../input/train_simplified/*.csv')
traindir = "../input/train_simplified/"

for index in range(0, len(class_paths)): # For every class.
    file = class_paths[index]
    
    if not file.endswith(".csv"):
        continue
    
    filename = file.split('.')[0]
    draw_df = None
    draw_df = pd.read_csv(file, nrows=50, skiprows=range(1, 500)) # Skip first 500 rows which were used for training

    size_of_set = len(draw_df)
    
    label = draw_df.word[0].replace(" ", "_")

    k = 0
    for i in range (0, size_of_set):
        if(draw_df.recognized[i] == False):
            continue
        file = "../my/test/" + label + "-" + str(i) + "-" + draw_df.countrycode[0]  + ".txt"
        f = open(file, "w")
        f.write(draw_df.drawing[i])
        f.close()
        k += 1
        if k == 15: # take 15 images per cat.
            break
   

In [ ]:
##################################### KERNEL#6
# This kernel shows 20 images from at least 8 classes here. Overall 160 images.

fig=plt.figure(figsize=(20, 120))
from matplotlib import pyplot, transforms

b = 0
drawinglist = []

files = os.listdir("../my/test")
files.sort()

image_count = 1
for file in files:
    
    F = open("../my/test/" + file,"r")
    
    drawing = F.read()
    drawing = ast.literal_eval(drawing)
    
    label = file.split('-')[0]

    
    if (drawinglist.count(label) == 20):
        continue
    
    if (len(set(drawinglist)) == 8 and drawinglist.count(label) == 0):
        break
    
    drawinglist.append(label)
    
    if image_count == 21:
        image_count = 1
    
    fig.add_subplot(32, 5, b + 1)
    for x, y in drawing: 
        # Rotate the picture because it's by default reversed.
        base = pyplot.gca().transData
        rot = transforms.Affine2D().rotate_deg(180)
        plt.plot(x, y,linewidth=12. , marker='+', transform= rot + base)
        plt.xticks(rotation=90)
        plt.title(str(image_count) + ". " + label)
        plt.axis('off')
    image_count += 1    
    b += 1
    
plt.show()

In [ ]:
##################################### KERNEL#7
# This kernel splits the data into train and val.

np.random.shuffle(train_grandarray)
split_x = int(len(train_grandarray) * 0.6) # Split train:val dataset 60:40

x_train = train_grandarray[0:split_x, 1:]
y_train = train_grandarray[0:split_x, 0]
x_val = train_grandarray[split_x: , 1:]
y_val = train_grandarray[split_x: , 0]

# todo left something for test as well

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)

y_train_onehot = keras.utils.to_categorical(y_train, num_classes)
y_val_onehot = keras.utils.to_categorical(y_val, num_classes)

print(x_train.shape, "\n",
      y_train_onehot.shape, "\n",
      x_val.shape, "\n",
      y_val_onehot.shape)

In [ ]:
##################################### KERNEL#8
# In this kernel a deep learning model is defined. 

from keras.layers.normalization import BatchNormalization

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dense(256, activation='relu')) # Remove then two of these
# model.add(Dense(256, activation='relu')) # Remove then two of these
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(256, activation='relu')) # Make this layer here twice so they will be 3
# #model.add(BatchNormalization())
# #model.add(Dropout(0.05)) # TODO remove one of these, try 0.9
# model.add(Dense(256, activation='relu')) # Remove then two of these
# #model.add(Dropout(0.05)) # TODO remove one of these, try 0.9
# model.add(Dense(256, activation='relu')) # Make this layer here twice so they will be 3
# model.add(Dropout(0.7)) # TODO remove one of these, try 0.9
# model.add(Dense(256, activation='relu')) # Remove then two of these

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.65))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

In [ ]:
##################################### KERNEL#9
# In this kernel the training is done and model evaluated on the validation set.

from keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor='acc', patience=3)]

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
batchsize = 16
model.fit(x_train, 
          y_train_onehot,
          batch_size = batchsize,
          epochs = 20,
          callbacks = callbacks,
          verbose = 1)

score = model.evaluate(x_val, y_val_onehot, verbose = 1, batch_size = batchsize)

# Clean up the memory
# x_train = None
# y_val_onehot = None
# y_train = None
# train_grandarray = None
# draw_df = None
# del x_train
# del y_val_onehot
# del y_train
# del train_grandarray
# del draw_df
# gc.collect()

print("\nAccuracy {}".format(score[1]))

In [ ]:
##################################### KERNEL#10
# In this kernel, the trained model is used to make predictions for the test dataset.
# Also, computes data used for calculating the accuracy by country.

from os import listdir
from os.path import isfile, join
import operator

# Load the test dataset
test_path = '../my/test'
test_files = [f for f in listdir(test_path) if isfile(join(test_path, f))]

#test_files.sort()

labels = {}
for l in range(0, len(reverse_dict)):
    labels[reverse_dict[l]] = 0

final_predictions = {}

success_collector = []
unsuccess_collector = []

country_dict = dict([ (elem, [0, 0]) for elem in country_list ]) # Convert Set to dict.

# Take all test files and make prediction in for loop to save RAM.
for i in range(0, len(test_files)):
    R = []
    F = open("../my/test/" + test_files[i],"r") 
    test_drawing = F.read()
    F.close()
    
    drawing = ast.literal_eval(test_drawing) 
    fig, ax = plt.subplots()
    for x,y in drawing:
        ax.plot(x, y, linewidth=12.)
    ax.axis('off')
    fig.canvas.draw()
    X = np.array(fig.canvas.renderer._renderer)
    plt.close("all")
    plt.clf()
    X = (cv2.resize(X, (img_rows, img_cols)) / 255.)[::-1]
    X = X[:, :, 3]         
    X = np.array(X)
    X = np.reshape(X, (img_rows, img_cols, 1))    
    R.append(X) # I have to append it like this so it fits the dimensions.
    
    R = np.array(R)
    prediction = model.predict_classes(R, batch_size=batchsize, verbose=0)
    
    X = None
    R = None
    del X
    del R
    gc.collect()

    groundtruth = test_files[i].split("-")[0]
    country = test_files[i].split("-")[2].split('.')[0]

    #print(str(i + 1) + ". Prediction for this image is: " + reverse_dict[prediction[0]] + ", Groundtruth is: " + groundtruth + ", Country: " + country) 
    
    # hardcode hotfix, I did notice issues with dash in the label names too late.
    if groundtruth == "teddy":
        groundtruth = "teddy-bear"
    if groundtruth == "t":
        groundtruth = "t-shirt"
    
    if reverse_dict[prediction[0]] == groundtruth:
        # Was classified correctly? increase the score.
        labels[groundtruth] += 1 # Increase score for computing category complexity.
        if (country not in country_dict):
            country_dict[country] = [0,0]
        country_dict[country][0] += 1 # First index of this dict represents number of correct guesses.
        if len(success_collector) < 5:
            success_collector.append([test_drawing, groundtruth, reverse_dict[prediction[0]]])
    else:
        # If classify incorrectly then decrese the score.
        labels[groundtruth] -= 1 # Decrease score for computing category complexity.
        if (country not in country_dict):
            country_dict[country] = [0,0]
        country_dict[country][1] += 1 # Second index of this dict represents number of incorrect guesses.
        if len(unsuccess_collector) < 5:
            unsuccess_collector.append([test_drawing, groundtruth, reverse_dict[prediction[0]]])
    
    if (i % 100 == 0):
        print(str(i) + " out of " + str(len(test_files)))
        
    #if i==50:
        #break
        
sorted_labels = sorted(labels.items(), key=operator.itemgetter(1))  
sorted_labels

In [ ]:
##################################### KERNEL#11
# This kernel finds top 5 categories where classification went wrong and where it wents well. 
# Lastly, it visualise 5 examples where prediction was correct and 5 where it failed.

print("Top most difficult categories to classify are: ")    
i = 0    
for key in sorted_labels:
    if(i < 5):
        print(str(i + 1) + ". " + key[0])
    else:
        break
    i +=1
        
print("Top most easiest categories to classify are: ")
j = 1
for i in range(len(sorted_labels) - 1, len(sorted_labels) - 6, -1):
    print(str(j) + ". " + sorted_labels[i][0])
    j += 1
    


fig = plt.figure(figsize=(15, 15))
for i in range(0, len(success_collector)):
    drawing = ast.literal_eval(success_collector[i][0])
    fig.add_subplot(5, 5, i + 1)
    for x, y in drawing: 
        base = pyplot.gca().transData
        rot = transforms.Affine2D().rotate_deg(180)
        plt.plot(x, y,linewidth=12. , marker='+', transform= rot + base)
        plt.xticks(rotation=90)
        plt.title(str(i + 1) + ". " + success_collector[i][1])
        plt.axis('off')

print("\n")
print("These are predictions labels for the second row (why weren't classfied correctly):")
fig = plt.figure(figsize=(15, 15))        
for i in range(0, len(unsuccess_collector)):
    drawing = ast.literal_eval(unsuccess_collector[i][0])
    fig.add_subplot(5, 5, i + 1)
    for x, y in drawing: 
        base = pyplot.gca().transData
        rot = transforms.Affine2D().rotate_deg(180)
        plt.plot(x, y,linewidth=12. , marker='+', transform= rot + base)
        plt.xticks(rotation=90)
        plt.title(str(i + 1) + ". " + unsuccess_collector[i][1])
        plt.axis('off')  
    print(str(i + 1) + ". " + unsuccess_collector[i][1] + " was classified as " + unsuccess_collector[i][2])

In [ ]:
##################################### KERNEL #12
# In this kernel, the accuracy by country is presented 

print("                         Accuracy by Country                  ")
print("___________________________________________________________________")
print("|Country |Number of guesses |Correct |Incorrect | Accuracy [%]|")
print("___________________________________________________________________")
for key, value in country_dict.items():
    number_of_guesses = value[0] + value[1]
    if number_of_guesses == 0:
        accuracy = 0
    else:
        accuracy = value[0] / number_of_guesses
    print("|   " + str(key) + "   |        " + str(number_of_guesses) + "         |    " +  str(value[0]) + "   |    " + str(value[1]) + "     |      " +  str(accuracy) + "      |")

Now, to the actual mini-project assignment:

1.

a.       How many images does the dataset consist of? 
		
		49707919 (Headers in .csv files were not counted)


b.       How many classes? How many images per class?

		340 classes. Number of images per class is shown as JSON in Kernel#3 


c.       Show 20 sample images from at least 8 different classes
		
		This is done in Kernel #6


d.       Consider if/how the data distribution will affect training of a classifier.

		I noticed that there is a noise which could cause some issues for my model. First of all, dataset is not evenly distributed. For example class airplane has ~10% more images than class alarm. This could present unwanted bias for the model. However, because of the memory limitation in Kaggle and the point that I haven't manage to make generator running in time before submition, I've decided to use fixed amount of samples for every class.
		Some of the images are empty or very bad drawn. Reason behind this is that in the quick draw game there is only 20 seconds to draw an image. While reviewing the .csv file I noticed that there is a column called "recognized" which contains boolean values. I've used only those which were recognized, in order to not use empty images, bad drawing and overall to decrease the noise which improved accuracy eventually.

2.       Use a network of your own choice (UNet, Classical CNN, ResNet, Faster R-CNN, etc.) to classify the dataset. You may take one of the existing kernels as point of departure. We expect that you document your experiences. Please experiment with:

		I have decided to use a variation of convnet - a deep-learning model almost universally used in CV applications. Convnet is basically stack of Conv2D and MaxPooling2D layers. First layer takes an input tensor of shape (image_height, image_width, image_channels) - not including the batch dimension. 
		Next the series of Conv2D and MaxPooling2D layers which output a 3D tensors of shape (height, width, channels). Width and hight dimensions shrinks as we go deeper to the network.
		The last step is to feed the output tensor into a densely connected classifier network.

--------------------------------------

a.       Hyper-parameters

		Training was done on 102 000 examples (300 images per category), validation dataset consisted of 68 000 images. Test set contained 14 960 images. This was later change because of Kaggle dying and going slow to 20 400 images for training and 13 600 for validation and 5 100.

		Learning rate - With low learning rates the loss curve will be close to linear line. Higher learning rates will decay the loss faster, but they get stuck at worse values of loss. Loss curve of higher learning rate will also look more exponencial.

		Firstly I have used Adam as an optimizer with learning rate 0.1. This didn't show up as an optimal solution because model stopped 

		learning rate 0.01 and model achieved 0.4470 acc on train and 0.3072 on validation set.

		Then I tried learning rate 0.005, which achieved slightly worse accuracy 0.3883/0.3563.

		The training time was still increasing so I was forced to decrese number of epochs to 20.
		Finally I set learning rate to 0.001 and result was 0.5921/0.4034, however I saw that after 20 epochs the network is still learning (acc was increasing and loss decreasing) so I re-run the training with 50 epochs and the result was 0.7364/0.3427, meaning that undesirable overfitting occured. 

		LR = 0.001 was chosen as the best one with 40 epochs.

		There are other optimizers to consider, such as Adamax, Nadam, SGDNesterov and so on. Adam was chosen because it has the best performance on MNIST dataset which is a similar CV problem.
		

b.       Batch sizes
			With higher batch sizes (512), the training goes much faster. However, it requires to iterate through much more epochs to get higher accuracy.
			The test was done with batch size 64. Then I tried following batch sizes:
			1. 64 - 0.7364/0.3427 40 epochs
			2. 512 - 0.6615/0.4075 40 epochs
			3. 128 - 0.6748/0.3736 40 epochs



			At this point I was forced to change my approach because the kernel was constantly dying in random times (https://www.kaggle.com/product-feedback/41221). I had to decrese number of images per category to just 100, so overall 340 000 images.
			Having batch size 32 and 40 epoch caused a huge overfitting 0.7/0.34. Thus I changed batch size back to 64 and put the training epochs to 20 achieving ~ 0.7 / ~ 0.25.



		

c.       Number of network layers, adding/removing batch normalization, increasing/decreasing dropout (if applicable)
		
		The overall architecture looked like this: Conv2D -> Maxpooling -> Conv2D -> Maxpooling -> Conv2D -> Flatten -> Dense -> Dense (sofmax).

		Droput layer takes care of removing neurons in order to make network generalize better and increase the validation accuracy. 
		Current accuracy is 0.7685/0.2561 In order to remove overfitting I will try to add a dropout layer before the last Dense layer with droput rate, 0.2 with result 0.4772/0.2868.

		I also tried to replace the dropout layer with another Dense layer with 256 neurons leading, as expected, to even more overfitting 0.8377/0.277.
		I have decided to keep the 256 neurons dense layer but after that add the droput layer with dropout rate 0.65 leading to 0.4375/0.2926. 

	
		Because of the data decrease I decreased batch normalization to 16 and I added one more droput layer with 0.2 rate, because still the training probability is much higher then validation one. Resulting in 0.30/0.273.

d.       Data augmentation techniques (random cropping, normalization, random erasing, etc.)

		I have found this paper about random erasing (https://arxiv.org/abs/1708.04896) from which I took some inspiration.
		Algorithm works as follows: Generates random probability in interval <0, 1>, if this probability is smaller then the erasing probability, then we are going to generate one white rectangle over the canvas.
		Rectangle with and high are randomly generated from an interval <20, 50>.
		Then random position is generated where the rectangle will be places. X axis from interval <1, 255> and Y from <1, 60>. 
		These intervals were manually estimated so the result is similar to examples in paper.
		Lastly, the canvas is merged utilizing matplotlib and numpy array acquired used in training.

		The acuraccy with augumentation probability 0 (no augumentation) was 0.7313/0.25 Then, I have tried to set the augumentation probability to 0.7 which led to accuraccy 0.6807/0.2553.

		Random erasing didn't show up as optimal augumentation technique in this situation. Probably because I'm working with very small number of data and there are many outliers in the dataset.
		It's worth noting that sometimes a generated rectangle did not erase any part of a drawing. This is because it was either too small or was positioned in a blank area.


3.       Break down the results of your classifier. Some suggestions:
		
		Reason why accuracy is not that high is that I have used only small amount of data for training (100* 0.6 images per category). By general rule of thumb it is reccomended to use at least 2000 samples per category. My intention was to use as much data as possible, but I was bounded with Kaggle RAM memory of 14 GB. I wanted to counter this by using a data generator, unfortunately my attemts failed. 

		In deep learning, having a more data is always better. Despite the Kaggle memory limits there are workarounds how to use as much data as possible. 
		Methods such as writing data to disk so they are not taking place in RAM, using generators instead of loading whole dataset at once, converting images into even smaller ones can be option as well. Then it is also possible to generalize better by creating new training images with data augmentation techniques such as (except mentioned above) flipping image verically or horizontally, rotating, zooming in or out, varying the color of an image. However, because of the time constrains and faulty behaviour of kaggle I couldn't make it on time.
		Model classifies well the drawing which are unique and can be easily distinguished from other drawings such as flower or Eifell tower. On the other hand drawings which share certain degree of similarity present a problem. These are for example Giraffe and horse which can be easily distinguished in real life by color, however in this particular drawing where you can draw just with one color it's very difficult.
		Also objects which have wheels, e.g. tractor and roller scates, pants and shorts.
		Then there were categories like which are basically subsets of the other one , for example bus and School bus.
		Furthermore, it classified violin as a guitar, cactus as a snake. Then classes which are very abstract were classified wrongly such as animal_migration and so on.

		To make neural network more accurate it is necessary to use more training data, identify outliers and see why were they classified incorrectly. It could be allowed to use colors in order to differentiate from school_bus and bus.

		A bit overfitting occured in my training. If I'd more time I would increase the dropout rates and increase number of epochs. Although, I used data augumentation technique, I use to to replace original data, not to expand the dataset as it was intended at first place.
		I should have use more training parameters. Also for tackling an overfitting a good technique is cross-validation.

a.       Which category is the easiest to classify? And the hardest? Did this correlate with the distribution of training data?

		Top most difficult categories to classify are (Number of samples in original dataset): 
		1. key (160 965) 
		2. marker (319 136)
		3. bridge (133 010)
		4. toothpaste (131 037)
		5. mosquito (123 029)

		Top most easiest categories to classify are (Number of samples in original dataset): 
		1. sailboat (136 506)
		2. circle (122 876)
		3. stairs (128 981)
		4. headphones (118 906)
		5. bowtie (130 283)

		My initial guess was that categories which have the most amount of guesses are easier to classify. This would have been because when a person doesn't know how to draw an object or it is very difficult, they might skip the round and move to drawing something else. However, my hypothesis wasn't confirmed. 


		For training, I made sure that every category has the same amount of samples (100*0.6), so different number of sample couldn't cause the network to be biased against some categories. Difference between categories which are easier to classify than the hard ones is that easy ones are just simple geometric shapes, whereas, the hard ones can be drawn from different angles, in different sizes and so on. 


b.       Are there any differences in classification accuracy from country to country?
		
		Table is done in Kernel#12.

		As it can be seen in the table not all the countries have same amount of guesses, some countries don't even have a guess.
		If we look only on those where number of guesses are higher than 250 there are US with 2490 and acc 0.30, GB with 390 and acc 0.29 and DE with 285 and acc 0.25.

		For making conclusions for this it is necessary to use much more data. From the results I got I can just conclude that most players are from US. 

c.       Visualize five examples where classification went well, and five where classification failed, consider what went wrong in case of the latter.
		
		Visualisation is done in KERNEL#11.


		Five examples where classification failed:

		1. squiggle was classified as animal_migration - This is because animal_migration was most of the time drawn just as some lines which should represent many animals going somewhere.
		2. pants was classified as pliers - I can see why, those pants actually look like pliers. 
		3. snowman was classified as pineapple - How can a carrot has the same size as whole snowman? This is pineapple. 
		4. airplane was classified as dragon - The backside of the airplane looks like dragon's had.
		5. piano was classified as jail - Those piano's keys look like jail bars.